In [1]:
import datasets
ds = datasets.load_dataset("livecodebench/code_generation_lite", split="test")
ds

/Users/federico.cassano/.pyenv/versions/3.9.19/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 400
})

In [2]:
ds[44]

{'question_title': 'find-maximum-number-of-string-pairs',
 'question_content': 'You are given a 0-indexed array words consisting of distinct strings.\nThe string words[i] can be paired with the string words[j] if:\n\nThe string words[i] is equal to the reversed string of words[j].\n0 <= i < j < words.length.\n\nReturn the maximum number of pairs that can be formed from the array words.\nNote that each string can belong in at most one pair.\n \nExample 1:\n\nInput: words = ["cd","ac","dc","ca","zz"]\nOutput: 2\nExplanation: In this example, we can form 2 pair of strings in the following way:\n- We pair the 0^th string with the 2^nd string, as the reversed string of word[0] is "dc" and is equal to words[2].\n- We pair the 1^st string with the 3^rd string, as the reversed string of word[1] is "ca" and is equal to words[3].\nIt can be proven that 2 is the maximum number of pairs that can be formed.\nExample 2:\n\nInput: words = ["ab","ba","cc"]\nOutput: 1\nExplanation: In this example, we 

In [3]:
import datetime
cutoff = datetime.datetime(2023, 9, 30, 0, 0)
ds = ds.filter(lambda ex: ex["contest_date"] >= cutoff)
ds

Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 202
})

In [4]:
import json
import pickle
import zlib
import base64


def decode_tests(tests):
    return json.loads(
                pickle.loads(
                    zlib.decompress(
                        base64.b64decode(tests)
                    )
                )
            )

In [5]:
# format we want:
# - question: has prompt
# - starter_code: has starter code, if any
# - difficulty: has difficulty
# - input_output: has tests, with fn_name key if needed
# - title: just for metadata
# - source: just for metadata
# - date: just for metadata
# - id: for unique id
cleaned_ds = []
for ex in ds:
    raw_tests = decode_tests(ex["private_test_cases"]) + json.loads(ex["public_test_cases"])
    tests = {"inputs": [], "outputs": []}
    metadata = json.loads(ex["metadata"])
    
    for test in raw_tests:
        inp = test["input"]
        out = test["output"]
        if "func_name" in metadata:
            inp = [json.loads(i) for i in inp.split("\n")]
            out = json.loads(out)
        tests["inputs"].append(inp)
        tests["outputs"].append(out)

    if "func_name" in metadata:
        name = metadata["func_name"]
        tests["fn_name"] = name
        
        
    
    obj = {
        "question": ex["question_content"],
        "starter_code": ex["starter_code"],
        "difficulty": ex["difficulty"],
        "input_output": json.dumps(tests),
        "title": ex["question_title"],
        "source": ex["platform"],
        "date": ex["contest_date"],
        "id": ex["question_id"],
    }
    cleaned_ds.append(obj)
    
cleaned_ds = datasets.Dataset.from_list(cleaned_ds)
# cleaned_ds.push_to_hub("cassanof/livecodebench_lite_filtered", split="test")

In [8]:
from codeprm.execution import check_executor_alive, smart_exec_tests
import json
print("Is server alive? " + str(check_executor_alive()))
print()
ex = cleaned_ds[44]
mock = ex["starter_code"] + "pass"
print(mock)
passing, err = smart_exec_tests(mock, json.loads(ex["input_output"]))
print(err)

Is server alive? True

class Solution:
    def countTestedDevices(self, batteryPercentages: List[int]) -> int:
        pass
1
Traceback (most recent call last):
  File "/tmp/codeexec/1.py", line 38, in <module>
    assert is_eq(Solution().countTestedDevices([30, 5, 0, 65, 50, 3, 60, 6, 50, 99]), 8)
AssertionError

Traceback (most recent call last):
  File "/tmp/codeexec/1.py", line 38, in <module>
    assert is_eq(Solution().countTestedDevices([30, 5, 0, 65, 50, 3, 60, 6, 50, 99]), 8)
AssertionError

